In [3]:
from bs4 import BeautifulSoup, NavigableString, Tag
import urllib
import requests
import re
import nltk
import time

import pandas as pd

In [4]:
scr = []
# 101,576
for ep in range(101,278):
    url = "http://www.chakoteya.net/NextGen/{}.htm".format(ep)
    try:
        scr.append(urllib.request.urlopen(url).read())
        time.sleep(3)
    except urllib.request.HTTPError as err:
        if err.code == 404:
            pass

In [5]:
many_soups = []
for ep in scr:
    many_soups.append(BeautifulSoup(ep, "lxml"))

In [7]:
many_soups[0].b.string

'Encounter\nat Farpoint'

In [8]:
sent_tokenizer = nltk.tokenize.sent_tokenize

In [9]:
pattern = re.compile(r'(\b[A-Z]+|([A-Z]+.[A-Z]+))(\:|\s\[.+\]\:)')

In [20]:
df = pd.DataFrame(columns=['character', 'ep_title', 'text'])

for ep in many_soups:
    
    episode_title = ep.b.string
    episode_title = episode_title.replace('\r\n', ' ')
    episode_title = episode_title.replace('\n', ' ')
    
    array_of_strings = []
    
    for string in ep.stripped_strings:
        string = string.replace('\r\n', ' ')
        string = string.replace('\n', ' ')
        array_of_strings.append(string)
        
    clean_df = []
    char_dict = {}

    for string in array_of_strings:
        found = re.search(pattern, string)
        if found is not None:
            stripped_string = string.replace(found.group(0), '').strip()
            stripped_string_tokenized = sent_tokenizer(stripped_string)

            key = found.group(1)

            for each in stripped_string_tokenized:
                    clean_df.append(each)
                    char_dict.setdefault(key, []).append(each)
    
    for key in char_dict:
        temp_df = pd.DataFrame(char_dict[key], columns=['text'])
        temp_df['character'] = key
        temp_df['ep_title'] = episode_title
        df = df.append(temp_df)

In [21]:
df.head()

,character,ep_title,text
0,PICARD,Encounter at Farpoint,"You will agree, Data, that Starfleet's orders ..."
1,PICARD,Encounter at Farpoint,As simple as that.
2,PICARD,Encounter at Farpoint,"It's hardly simple, Data, to negotiate a frien..."
3,PICARD,Encounter at Farpoint,"Data, how can you be programmed as a virtual e..."
4,PICARD,Encounter at Farpoint,"It means to spy, to sneak."


In [22]:
df.size

326583

In [26]:
df.to_csv('dataframe_all_tng.csv', index=False)

In [27]:
df_test = pd.read_csv('./dataframe_all_tng.csv')

In [28]:
df_test.head()

,character,ep_title,text
0,PICARD,Encounter at Farpoint,"You will agree, Data, that Starfleet's orders ..."
1,PICARD,Encounter at Farpoint,As simple as that.
2,PICARD,Encounter at Farpoint,"It's hardly simple, Data, to negotiate a frien..."
3,PICARD,Encounter at Farpoint,"Data, how can you be programmed as a virtual e..."
4,PICARD,Encounter at Farpoint,"It means to spy, to sneak."
